In [34]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from collections import Counter
from os.path import join as oj

import numpy as np
from matplotlib import pyplot as plt
plt.style.use('dark_background')
import pandas as pd
import pickle as pkl
from tqdm import tqdm
from style import *
out_dir = 'results/aug6_6/' # jun17_1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# compare all models

In [35]:
def load_results(out_dir):
    r = []
    for fname in tqdm(os.listdir(out_dir)):
        d = pkl.load(open(oj(out_dir, fname), 'rb'))
        d['param_name'] = fname[:-4]
        r.append(pd.Series(d))
    r = pd.concat(r, axis=1, sort=False).T.infer_objects()
    r = r.reindex(sorted(r.columns, reverse=True), axis=1) # sort the column names
    r = r.round(3)
    r = r.set_index('param_name')
    return r

results = load_results(out_dir)
# print(results.keys())

100%|██████████| 117/117 [00:00<00:00, 232.53it/s]


## look at prediction metrics

Most important thing is to have very low false negatives (very close to zero). After that, we also want to minimize the false positives.

In [36]:
# prev rule: sensitivity 0.97, specificity 0.425
r = results
# print(r.keys())
# r = r[[k for k in r if not 'std' in k]]
# r = r[[k for k in r if not '_test' in k]]
# r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
# r = r.sort_values(by=['balanced_accuracy'], ascending=False)

r = r[r['sensitivity_cv'] > 0.8] # what fraction of true pos do we return?
r = r[r['specificity_cv'] > 0.4] # what fraction of true negs do we return?

r = r.sort_values(by=['sensitivity_cv', 'specificity_cv'], ascending=False)
# r = r[['fn', 'tn', 'fp', 'tp', 'specificity', 'sensitivity', 'roc_auc', 'precision',
#        'f1', 'balanced_accuracy', 'accuracy']]
r = r[['sensitivity_cv', 'specificity_cv', # 'precision',
       'sensitivity_test1', 'specificity_test1',
       'sensitivity_test2', 'specificity_test2',
       'fn_cv', 'tn_cv', 'fp_cv', 'tp_cv',
       'precision_cv', 'roc_auc_cv', 
#        'f1_cv', 'balanced_accuracy_cv', 'accuracy_cv',
       'fn_test1', 'tn_test1', 'fp_test1', 'tp_test1',
       'fn_test2', 'tn_test2', 'fp_test2', 'tp_test2']]
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
# r = r[r.index == 'logistic_select_lasso=27_ros=100']
# r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r

,sensitivity_cv,specificity_cv,sensitivity_test1,specificity_test1,sensitivity_test2,specificity_test2,fn_cv,tn_cv,fp_cv,tp_cv,precision_cv,roc_auc_cv,fn_test1,tn_test1,fp_test1,tp_test1,fn_test2,tn_test2,fp_test2,tp_test2
param_name,,,,,,,,,,,,,,,,,,,,
logistic_select_stab_lasso=6_sample_weights=500,0.969,0.522,0.877,0.513,0.984,0.507,4,4129,3774,126,0.032,0.898,9,2021,1917,64,1,1077,1049,61
logistic_select_stab_lasso=7_sample_weights=750,0.969,0.509,0.890,0.493,0.968,0.496,4,4026,3877,126,0.031,0.896,8,1943,1995,65,2,1055,1071,60
logistic_select_stab_lasso=6_sample_weights=750,0.969,0.504,0.904,0.484,0.984,0.476,4,3983,3920,126,0.031,0.897,7,1905,2033,66,1,1011,1115,61
logistic_select_stab_lasso=6_sample_weights=1000,0.969,0.504,0.904,0.484,0.984,0.476,4,3983,3920,126,0.031,0.897,7,1905,2033,66,1,1011,1115,61
logistic_select_stab_lasso=7_sample_weights=500,0.962,0.546,0.863,0.587,0.952,0.556,5,4312,3591,125,0.034,0.897,10,2312,1626,63,3,1182,944,59
logistic_select_stab_lasso=5_sample_weights=500,0.954,0.531,0.863,0.524,0.984,0.516,6,4193,3710,124,0.032,0.884,10,2063,1875,63,1,1097,1029,61
dt_select_stab_lasso=5_sample_weights=500,0.885,0.543,0.808,0.533,0.855,0.539,15,4289,3614,115,0.031,0.819,14,2098,1840,59,9,1146,980,53
dt_select_stab_lasso=5_sample_weights=750,0.885,0.407,0.932,0.013,0.871,0.023,15,3217,4686,115,0.024,0.819,5,52,3886,68,8,49,2077,54
logistic_select_stab_lasso=5_sample_weights=100,0.862,0.770,0.781,0.763,0.919,0.687,18,6086,1817,112,0.058,0.886,16,3003,935,57,5,1460,666,57


**look at feat importances**

In [45]:
best_model = 'logistic_select_stab_lasso=6_sample_weights=500' #logistic_select_rf=5_smote=5'

results.keys()

Index(['tp_test2', 'tp_test1', 'tp_cv_folds', 'tp_cv', 'tn_test2', 'tn_test1',
       'tn_cv_folds', 'tn_cv', 'specificity_test2', 'specificity_test1',
       'specificity_cv_folds', 'specificity_cv', 'sensitivity_test2',
       'sensitivity_test1', 'sensitivity_specificity_curve_test2',
       'sensitivity_specificity_curve_test1',
       'sensitivity_specificity_curve_cv_folds',
       'sensitivity_specificity_curve_cv', 'sensitivity_cv_folds',
       'sensitivity_cv', 'roc_curve_test2', 'roc_curve_test1',
       'roc_curve_cv_folds', 'roc_curve_cv', 'roc_auc_test2', 'roc_auc_test1',
       'roc_auc_cv_folds', 'roc_auc_cv', 'precision_test2', 'precision_test1',
       'precision_recall_curve_test2', 'precision_recall_curve_test1',
       'precision_recall_curve_cv_folds', 'precision_recall_curve_cv',
       'precision_cv_folds', 'precision_cv', 'model_type', 'model_best',
       'metrics', 'imps_best', 'idx_cv_best', 'fp_test2', 'fp_test1',
       'fp_cv_folds', 'fp_cv', 'fn_test2', 

In [43]:
r = results
best_model = 'logistic_select_stab_lasso=6_sample_weights=500' #logistic_select_rf=5_smote=5'
# print(r.keys())
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
# r = r.sort_values('roc_auc_cv', ascending=False)
# keys = [k for k in r if '_f' in k]
# keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
# r = r[r.index.str.contains('35')]
# r = r[r.index.str.contains('11')]
# r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})

# r = r[r.index.str.contains('20')]
r = r[r.index.str.contains(best_model)]

r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r

# r = rank(r)
# r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank
# idxs = r.index
# r.insert(0, 'balanced_acc', results.loc[idxs]['balanced_accuracy'])
# r = r.sort_values('balanced_acc', ascending=False)
# subset = list(r.keys())
# subset.remove('balanced_acc')
# r.to_html('table.html')
# r = r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1) #, subset=subset) # rows differently colored


r = r.transpose()
# r = r.reindex(r[best_model].abs().sort_values(ascending=False).index) # sort by abs value
# r = r.sort_values(by='logistic_20_ros=1')
r = r.fillna(0).style.background_gradient(cmap=cm) #, axis=0) #, subset=subset) # rows differently colored
# r = background_gradient(r)
r

# misc analysis

In [ ]:
def count_common_feats(m_cv, model_type='tree'):
    if model_type == 'tree':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            feats_used += list(ks[m_fit.feature_importances_ != 0])
    elif model_type == 'logistic':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            num_feats = 5
            # get top num_feats features with biggest bsolute weights
            idxs = np.abs(m_fit.coef_).flatten().argsort()[-num_feats:][::-1]
            feats_used += list(ks[idxs])
    return sorted(dict(Counter(feats_used)).items(), key=lambda kv: kv[1], reverse=True)
        
count_common_feats(m_cv, model_type)

In [ ]:
# plot a tree
m_fit = m_cv['estimator'][0]
plt.figure(dpi=300)
plot_tree(m_fit, feature_names=ks)
plt.show()